In [1]:
# %%capture
!pip install pydub
!pip install noisereduce
!pip install pywt
!pip install json-tricks
!pip install datasets
!pip install transformers
!pip install streamlit==0.84.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pywt (from versions: none)
ERROR: No matching distribution found for pywt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 7.7 MB/s 
     |████████████████████████████████| 140 kB 68.4 MB/s 
     |████████████████████████████████| 1.1 MB 67.3 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 212 kB 77.5 MB/s 
     |███████████████████

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchaudio
from tqdm import tqdm
from pathlib import Path

In [ ]:
# %%capture
!gdown --id 1cM4aGnx_4C_8USaHpZJOhJf3bX_6nBAd
!unzip /content/archive.zip
!mkdir -p RAVDESS
!gdown --id 1ijquqK2v2MPXrO9NFlr7QkhMEazAgult
!unzip /content/RAVDESS.zip -d /content/RAVDESS

In [5]:
#get audio path
def get_wavpath(name):
  wavlist = []
  for subdir, dirs, files in os.walk(name):
    for file in files: 
      path = os.path.join(subdir.split('/')[2],file)
      wavlist.append(path)
  return wavlist 

In [ ]:
for subdir, dirs, files in os.walk('/content/RAVDESS'):
    print("subdir: ", subdir)
    print("dir: ", dirs)
    print("files: ", files)
    for file in files: 
      path = os.path.join(subdir,file)
      break
    break

subdir:  /content/RAVDESS
dir:  ['Actor_14', 'Actor_05', 'Actor_10', 'Actor_09', 'Actor_11', 'Actor_24', 'audio_speech_actors_01-24', 'Actor_08', 'Actor_15', 'Actor_07', 'Actor_19', 'Actor_22', 'Actor_04', 'Actor_06', 'Actor_18', 'Actor_03', 'Actor_21', 'Actor_12', 'Actor_02', 'Actor_20', 'Actor_23', 'Actor_01', 'Actor_13', 'Actor_16', 'Actor_17']
files:  []


In [6]:
ravd = []
for path in tqdm(Path("RAVDESS").glob("**/*.wav")):
    try:
        # There are some broken files
        s = torchaudio.load(path)
        ravd.append(
            str(path)
        )
    except Exception as e:
        # print(str(path), e)
        pass


2880it [00:04, 644.99it/s]


In [7]:
ravd[0]

'RAVDESS/Actor_19/03-01-04-02-01-02-19.wav'

In [ ]:
check = []
for p in ravd: 
    try:
        s = torchaudio.load(p)
        check.append(str(p))
    except Exception as e:
        print(e)

In [8]:
#get labels 

ravd_labels = []


for path in ravd: 
  lab = path.split('/')[-1].split('-')[2]
  if lab == '01':
    ravd_labels.append('neutral')
  elif lab == '02':
    ravd_labels.append('calm')
  elif lab == '03':
    ravd_labels.append('happy')
  elif lab == '04':
    ravd_labels.append('sad')
  elif lab == '05':
    ravd_labels.append('angry')
  elif lab == '06':
    ravd_labels.append('fearful')
  elif lab == '07': 
    ravd_labels.append('disgusted')
  elif lab == '08':
    ravd_labels.append('surprised')

ravd_labels[3]

'sad'

In [9]:
tess = []
for path in tqdm(Path("TESS Toronto emotional speech set data").glob("**/*.wav")):
    try:
        # There are some broken files
        s = torchaudio.load(path)
        tess.append(
            str(path)
        )
    except Exception as e:
        # print(str(path), e)
        pass
#get labels 

tess_labels = []

for path in tess: 
  lab = path.split('/')[-1].split('_')[2].split('.')[0]
  if lab == 'neutral':
    tess_labels.append('neutral')
  elif lab == 'happy':
    tess_labels.append('happy')
  elif lab == 'sad':
    tess_labels.append('sad')
  elif lab == 'angry':
    tess_labels.append('angry')
  elif lab == 'fear':
    tess_labels.append('fearful')
  elif lab == 'disgust': 
    tess_labels.append('disgusted')
  elif lab == 'ps': 
    tess_labels.append('surprised') 

tess_labels[5]
tess[5]

2800it [00:01, 1974.45it/s]


'TESS Toronto emotional speech set data/YAF_sad/YAF_tell_sad.wav'

In [10]:
#Dataframe 2 datasets
# data = pd.DataFrame(np.column_stack([tess, tess_labels]),
#                            columns=['Path', 'Emotion'])
ravd_frame = pd.DataFrame(np.column_stack([ravd, ravd_labels]),
                           columns=['Path', 'Emotion'])

tess_frame = pd.DataFrame(np.column_stack([tess, tess_labels]),
                          columns=['Path', 'Emotion'])

data = pd.concat([ravd_frame, tess_frame],
                 axis=0, ignore_index=True)

#save .csv
data.to_csv('Dataset.csv', header=True)

data = pd.read_csv('Dataset.csv', index_col=0)

In [11]:
data

,Path,Emotion
0,RAVDESS/Actor_19/03-01-04-02-01-02-19.wav,sad
1,RAVDESS/Actor_19/03-01-05-01-02-02-19.wav,angry
2,RAVDESS/Actor_19/03-01-04-01-02-01-19.wav,sad
3,RAVDESS/Actor_19/03-01-04-02-02-02-19.wav,sad
4,RAVDESS/Actor_19/03-01-04-02-01-01-19.wav,sad
...,...,...
5675,TESS Toronto emotional speech set data/YAF_fea...,fearful
5676,TESS Toronto emotional speech set data/YAF_fea...,fearful
5677,TESS Toronto emotional speech set data/YAF_fea...,fearful
5678,TESS Toronto emotional speech set data/YAF_fea...,fearful


In [12]:
path = data['Path'].to_list()
emotion = data['Emotion'].to_list()

In [14]:
print(path[-1])
print(emotion[-1])

TESS Toronto emotional speech set data/YAF_fear/YAF_live_fear.wav
fearful


In [13]:
print(f"Step 0: {len(data)}")

data["status"] = data["Path"].apply(lambda path: True if os.path.exists(path) else None)
data = data.dropna(subset=["Path"])
data = data.drop("status", 1)
print(f"Step 1: {len(data)}")

data = data.sample(frac=1)
data = data.reset_index(drop=True)
data

Step 0: 5680
Step 1: 5680


,Path,Emotion
0,TESS Toronto emotional speech set data/OAF_ang...,angry
1,RAVDESS/audio_speech_actors_01-24/Actor_05/03-...,fearful
2,TESS Toronto emotional speech set data/OAF_dis...,disgusted
3,TESS Toronto emotional speech set data/YAF_neu...,neutral
4,RAVDESS/audio_speech_actors_01-24/Actor_20/03-...,surprised
...,...,...
5675,TESS Toronto emotional speech set data/YAF_ple...,surprised
5676,TESS Toronto emotional speech set data/YAF_ple...,surprised
5677,TESS Toronto emotional speech set data/OAF_Ple...,surprised
5678,TESS Toronto emotional speech set data/OAF_Ple...,surprised


In [15]:
print("Labels: ", data["Emotion"].unique())
print()
data.groupby("Emotion").count()[["Path"]]

Labels:  ['angry' 'fearful' 'disgusted' 'neutral' 'surprised' 'happy' 'sad' 'calm']



,Path
Emotion,
angry,784
calm,384
disgusted,784
fearful,784
happy,784
neutral,592
sad,784
surprised,784


In [16]:
!mkdir -p data

In [17]:
from sklearn.model_selection import train_test_split


In [18]:
save_path = "/content/data"

train_df, test_df = train_test_split(data, test_size=0.2, random_state=101, stratify=data["Emotion"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)

(4544, 2)
(1136, 2)


In [19]:
# Loading the created dataset using datasets
from datasets import load_dataset, load_metric


data_files = {
    "train": "/content/data/train.csv", 
    "validation": "/content/data/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Using custom data configuration default-ccdfc82eac93b70c


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ccdfc82eac93b70c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['Path', 'Emotion'],
    num_rows: 4544
})
Dataset({
    features: ['Path', 'Emotion'],
    num_rows: 1136
})


In [20]:
# We need to specify the input and output column
input_column = "Path"
output_column = "Emotion"

In [21]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 8 classes: ['angry', 'calm', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [22]:
from transformers import AutoConfig, Wav2Vec2ForCTC, Wav2Vec2Processor

In [23]:
# model_name_or_path = "lighteternal/wav2vec2-large-xlsr-53-greek"
pooling_mode = "mean"
model_name_or_path = "facebook/wav2vec2-base-960h"

In [24]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

In [25]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The target sampling rate: 16000


In [ ]:
print(processor.feature_extractor.feature_size)
print(processor.feature_extractor.padding_value)
print(processor.feature_extractor.do_normalize)
print(processor.feature_extractor.return_attention_mask)

1
0.0
True
False


In [26]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array[0]).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [27]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=32,
    batched=True,
    num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=32,
    batched=True,
    num_proc=4
)
# train_dataset = train_dataset.map(preprocess_function,num_proc=4)
# eval_dataset = eval_dataset.map(preprocess_function, num_proc=4)

#0:   0%|          | 0/36 [00:00<?, ?ba/s]

#1:   0%|          | 0/36 [00:00<?, ?ba/s]

#2:   0%|          | 0/36 [00:00<?, ?ba/s]

#3:   0%|          | 0/36 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

#1:   0%|          | 0/9 [00:00<?, ?ba/s]

#0:   0%|          | 0/9 [00:00<?, ?ba/s]

#2:   0%|          | 0/9 [00:00<?, ?ba/s]

#3:   0%|          | 0/9 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

In [28]:
train_dataset

Dataset({
    features: ['Path', 'Emotion', 'input_values', 'labels'],
    num_rows: 4544
})

In [29]:
!mkdir -p RAV_TESS_preprocessed_data

In [30]:
from datasets import DatasetDict, load_from_disk

dataset = DatasetDict({
    "train": train_dataset, 
    "validation": eval_dataset
})
dataset.save_to_disk("/content/RAV_TESS_preprocessed_data")

In [31]:
!zip -r RAV_TESS_preprocessed_data.zip /content/RAV_TESS_preprocessed_data

  adding: content/RAV_TESS_preprocessed_data/ (stored 0%)
  adding: content/RAV_TESS_preprocessed_data/dataset_dict.json (stored 0%)
  adding: content/RAV_TESS_preprocessed_data/train/ (stored 0%)
  adding: content/RAV_TESS_preprocessed_data/train/dataset_info.json (deflated 66%)
  adding: content/RAV_TESS_preprocessed_data/train/dataset.arrow (deflated 15%)
  adding: content/RAV_TESS_preprocessed_data/train/state.json (deflated 41%)
  adding: content/RAV_TESS_preprocessed_data/validation/ (stored 0%)
  adding: content/RAV_TESS_preprocessed_data/validation/dataset_info.json (deflated 66%)
  adding: content/RAV_TESS_preprocessed_data/validation/dataset.arrow (deflated 15%)
  adding: content/RAV_TESS_preprocessed_data/validation/state.json (deflated 41%)


In [33]:
!cp -av /content/RAV_TESS_preprocessed_data.zip /content/drive/MyDrive/CN7/DLP/voice_emotion_recognition/dataset

'/content/RAV_TESS_preprocessed_data.zip' -> '/content/drive/MyDrive/CN7/DLP/voice_emotion_recognition/dataset/RAV_TESS_preprocessed_data.zip'


In [ ]:
type(train_dataset[0]['labels'])

int

In [ ]:
print(train_dataset[0])

{'Path': 'TESS Toronto emotional speech set data/YAF_angry/YAF_pad_angry.wav', 'Emotion': 'angry', 'input_values': [-0.00023985425650607795, -0.00045382874668575823, -0.0092603275552392, -0.014003465883433819, -0.004002446308732033, -0.0052757239900529385, 0.005465683992952108, -0.0007355363923124969, -0.005527231842279434, -0.0008071066695265472, -0.006467579863965511, -0.007309836335480213, -0.004177781753242016, 0.006345209665596485, 0.0015100232558324933, 0.002221031580120325, -0.002033822238445282, -0.001488191308453679, -0.008904659189283848, -0.01905427686870098, -0.002488452475517988, 0.003974767867475748, 0.0005550239002332091, 0.0019335892284289002, -0.0008255259599536657, -0.0049464828334748745, -0.0015376543160527945, -0.015074362978339195, -0.006154356990009546, -0.004311486147344112, -0.011398103088140488, -0.000503760704305023, -0.008835463784635067, -0.002742960350587964, -0.009100453928112984, -0.007604140322655439, 0.003599372459575534, 0.007447260897606611, 0.0063115

In [ ]:
type(train_dataset[0]['input_values'])

list

In [ ]:
speechs = []
for a in tqdm(train_dataset['Path']): 
    arr, sr = torchaudio.load(a)
    if arr.size(0) != 1: 
        speechs.append((a, arr.size()))

100%|██████████| 2240/2240 [00:01<00:00, 2180.74it/s]


In [ ]:
len(speechs)

0